Берем текст, разбиваем его на предложения.

В каждом предложении берем эмбеддинги слов и из них собираем общий эмбеддинг для предложения. 

Гипотеза: если предложения относятся к одной теме, то их вектора этих предложений будут похожи.

Для этого мы считаем косинусную меру сходства между 1-ым и 2-м предложением текста, 2-м и 3-м и так далее.

Выбираем некоторый порог (допустим, 0.75), и если cosine similarity выше этого порога - объединяем предложения в один кусок.



In [2]:
import gensim
#import nltk
#nltk.download('punkt')
from string import punctuation
from collections import Counter
import numpy as np
import re
import os
from nltk import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [3]:
path = '../papaver_main_parts'
data = []
for item in os.walk(path):
    for file in item:
        path_file = path + os.sep + file
        f = open(str(path_file), 'r', encoding='utf-8')
        lines = [line for line in f.readlines()]
        data.append(lines)

In [4]:
data_files = ['C:/Users/User/papaver_main_parts/'+file for file in os.listdir('C:/Users/User/papaver_main_parts')  if 'main' in file]

In [5]:
len(data_files)

653

In [6]:
data = ''
for data_file in data_files:
    with open(str(data_file), 'r', encoding='utf-8') as infile:
        for line in infile:
            data += line

In [7]:
print(data[:91])

УСТАНОВИЛ:
 Мышанский С.И., имея умысел на незаконное приобретение, хранение и изготовление


In [8]:
punct = punctuation+'«»—…“”*№–'
def normalize(text):
    """Принимает строку, возвращает список строк.
       Разбивает текст по пробельным символам, приводит к нижнему регистру
       и удаляет пунктуацию на концах токенов.
       
    """
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word]


    return words

def get_text_embedding(text):
    """Принимает строку, возвращает np.array.
       Нормализует текст, создает нграммы, вытакскивает вектора из модели,
       усредняет вектора отдельных слов.
       
    """
    text_normalized = p2[p[normalize(text)]]
    
    # чтобы не обрабатывать одно и то же слово несколько раз
    # сделаем счетчик, частоту слов используем как вес
    words = Counter(text_normalized)
    # длина текста для нормализации частоты
    total_num_words = len(text_normalized)
    
    vecs = np.zeros((len(words), 200))
    weights = []
    
    for i, word in enumerate(words):
        try:
            v = fast_text.wv.get_vector(word)
        # иногда слова нет в модели, тогда приписываем слову
        # нулевой вес (т.е не учитываем его при усреднении)
        except (KeyError, ValueError):
            weights.append(0)
            continue
        
        vecs[i] = v
        # каждому слову приписывается вес (частота/длину текста в токенах)
        weights.append((words[word]/total_num_words))
    
    # если какие-то слова из текста есть в модели
    # выдаем усредненный вектор
    if any(weights):
        text_embedding = np.average(vecs, axis=0, weights=weights)
        return text_embedding
    
    # если нет, выдаем нули
    else:
        return np.zeros((1, 200))

In [9]:
# собираем статистики
ph = gensim.models.Phrases(data)

In [10]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

In [11]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[data])
p2 = gensim.models.phrases.Phraser(ph2)

In [12]:
p2[p[data[40:66]]]

['н',
 'а',
 ' ',
 'н',
 'е',
 'з',
 'а',
 'к',
 'о',
 'н',
 'н',
 'о',
 'е',
 ' ',
 'п',
 'р',
 'и',
 'о',
 'б',
 'р',
 'е',
 'т',
 'е',
 'н',
 'и',
 'е']

In [16]:
fast_text = gensim.models.FastText(data, size=200, min_n=4, max_n=8)

In [17]:
text = """ Рякин Д.А., совершил незаконное изготовление, хранение без цели сбыта наркотических средств в крупном размере.
Преступления совершено им при следующих обстоятельствах.
Рякин Д.А., 26 февраля 2012 года около 14 часов, находясь в летней кухне <адрес>, из ранее приобретенных им семян мака и растворителя, умышленно, незаконно в нарушении требования ФЗ № 3 от 08.01.1998 года «О наркотических средствах и психотропных веществах», без цели сбыта, ..., изготовил средство, которое согласно заключений эксперта, является наркотическим средством - экстрактом маковой соломы, массой не менее 1,29 грамма сухого вещества, размер которого согласно утвержденного Постановления Правительства Российской Федерации № 76 от 07.02.2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, для целей статей 228, 228.1 и 229 Уголовного кодекса Российской Федерации», является крупным размер. В тот же день часть незаконно изготовленного наркотического средства - экстракт маковой соломы Рякин Д.А. употребил ..., а оставшуюся часть наркотического средства - экстракт маковой соломы массой 1,29 грамма, что относится к крупному размеру, умышленно, незаконно, без цели сбыта, для собственного употребления, хранил в летней кухне домовладении <адрес> до 14 часов 40 минут 26 февраля 2012 года.
Подсудимый Рякин Д.А., в судебном заседании вину в совершенном преступлении признал, пояснил, что он ..., изготовленные из семян мака, ..., но не систематически, а периодически с 2008 года. Дома наркотические средства он изготавливать не может, так как проживает в данный момент сожительницей К.С.В. и их малолетним ребенком ДД.ММ.ГГГГ года рождения. 26.02.2012 года утром он употребил дома водку в количестве 250 грамм и после этого ему захотелось употребить наркотическое средство, в этот же день он, находясь в г. Апшеронске на рынке у незнакомой ему женщины около 13 часов он приобрел два пакетика, общей массой 200 грамм семян растения мак, в заводской упаковке, с надписью «Мак кондитерский», в хозяйственном магазине купил бутылку ..., в аптеке приобрел медицинский шприц и пошел в сторону <адрес>, где проживает его знакомый Н.В.П. Около 13 часов 45 минут он пришел к Н.В.П. домой, прошел в летнюю кухню, расположенную во дворе домовладения <адрес>, отдал ему бутылку пива. После у Н.В.П. взял металлический ковш и стал изготавливать в летней кухне наркотическое средство экстракт маковой соломы из принесенных им семян растения мак и ..., .... .... Примерно через 20 минут он услышал, что кто-то зовет хозяина дома, он позвал Н.В.П. и сказал ему, что его кто-то зовет, Н.В.П. вышел на улицу, а он остался в летней кухне. Через некоторое время в летнюю кухню зашли сотрудники полиции с Н.В.П. и двое незнакомых ему людей. Сотрудники полиции спросили, кто он такой и попросили у него документы удостоверяющие личность, представившись, Рякин пояснил, что документов у него при себе нет. Сотрудники полиции в присутствии двух понятых задали ему вопрос, имеются ли у него запрещенные в свободном обороте вещества, наркотические средства, предметы, добытые преступным путем и если таковы имеются, предложили ему их добровольно выдать, на что он ответил, что ничего у него нет и выдавать ему нечего. После чего сотрудниками полиции была осмотрена летняя кухня, в присутствии двух понятых на кухне на столе был обнаружен использованный одноразовый медицинский шприц, а рядом обрезанная пластиковая бутылка, в которой находились семена мака с резким запахом ..., в левом углу на полу находилась бутылка из полимерного вещества емкостью 1,5 литра с жидкостью светло-коричневого цвета с резким запахом ..., на веранде в летней кухне на газовой плите находился эмалированный ковш в котором находилась жидкость коричневого цвета, а рядом на подоконнике находилась пустая бутылка из-под ..., после чего для удобства транспортировки данная жидкость из эмалированного ковша была перелита в бутылку. Все перечисленные предметы были изъяты, помещены в три полиэтиленовых пакета, горловины которых были ошнурованы нитью, концы которых были опечатаны печатью № 66 «для пакетов» отдела МВД России по Апшеронскому району, где он и двое понятых поставили свои подписи. На вопрос сотрудника полиции, что за вещество в бутылках он пояснил, что Н.В.П. к ним никакого отношения не имеет, а данные предметы принадлежат ему и что в них находится наркотическое средство, которое он изготовил сам из семян мака и растворителя для собственного употребления без цели сбыта. О том, что изготовление, хранение наркотических средств запрещено законом он знал. 
Оценивая все доказательства в их совокупности, суд находит, что вина подсудимого Рякина Д.А. в совершенном им преступлении установлена в судебном заседании и подтверждается следующими доказательствами.
 В судебном заседании по ходатайству государственного обвинителя, с согласия подсудимого, адвоката, в связи с неявкой в суд свидетеля Х.А.А., в соответствии со ст. 281 УПК РФ оглашен протокол его допроса в качестве свидетеля от 12.03.2012 года (л.д. 35-37), из которого следует, что 26.02.2012 года около 14 часов 30 минут он находился в районе <адрес>, его остановили сотрудники полиции и пригласили в качестве понятого при осмотре домовладения <адрес>, после чего они пригласили еще одного понятого и разъяснили им права и обязанности понятых. После чего сотрудник полиции пояснил, что в Дежурную Часть Отдела МВД России по Апшеронскому району поступило сообщение, что по данному адресу собрались наркоманы и изготавливают наркотические средства. В его присутствии и в присутствии другого понятого сотрудник полиции позвал хозяина домовладении, вышел пожилой мужчина, представился Н.В.П., сотрудник полиции пояснил, что в Дежурную Часть Отдела МВД России по Апшеронскому району поступило сообщение, что по данному адресу собрались наркоманы и изготавливают наркотические средства. Н.В.П. пояснил, что ничего подобного в его домовладении не происходит и предложил пройти и убедиться самим. После чего Х.А.А., другой понятой и сотрудники полиции прошли в летнюю кухню домовладения <адрес>. Перед началом осмотра в их присутствии сотрудник полиции поинтересовался у гр. Н.В.П. имеются ли у него запрещенные в свободном обороте предметы, наркотические средства, боевые припасы и если таковые имеются предложил их добровольно выдать, разъяснил, что в случае добровольной выдачи он будет освобожден по уголовной ответственности, на что Н.В.П. пояснил, что ничего запрещенного у него нет и выдать ему нечего. В летней кухне также находился молодой мужчина, который представился знакомым Н.В.П., Рякиным Д.А., сотрудник полиции также в его присутствии и в присутствии другого понятого поинтересовался у Рякина Д.А., имеются ли у него запрещенные в свободном обороте предметы, наркотические средства, боевые припасы и если таковые имеются предложил их, добровольно выдать, разъяснил, что в случае добровольной выдачи он будет освобожден от уголовной ответственности. Рякин Д.А. пояснил, что ничего запрещенного у него нет. В ходе осмотра летней кухни в его присутствии и присутствии другого понятого, Н.В.П., Рякина Д.А. в кухне на столе был обнаружен одноразовый медицинский шприц, со следами коричневого цвета на внутренней поверхности, рядом со шприцем на столе была обнаружена обрезанная пластиковая бутылка, в которой находились семена растения мак с резким запахом растворителя. На полу в верхнем углу находилась бутылка из полимерного материала, емкостью 1,5 литра с жидкостью коричневого цвета с резким запахом ..., на веранде на газовой плите находился эмалированный ковш, в котором находилась жидкость коричневого цвета с резким запахом ... рядом с плитой на подоконнике находилась стеклянная бутылка из-под ... На вопрос сотрудников полиции Рякин Д.А. пояснил, что в ковше и в бутылке находится наркотическое средство экстракт маковой соломы, которое он изготовил сам из принесенных им семян мака и ..., для собственного употребления без цели сбыта. После чего сотрудниками полиции, в его присутствии, в присутствии другого понятого Н.В.П., Рякина Д.А. жидкость из ковша, для удобства транспортировки, была перелита в стеклянную бутылку из-под ... и изъята, так же была изъята бутылка с семенами растения мак, пластиковая бутылка с жидкостью коричневого цвета, медицинский шприц и ковш. Все изъятые предметы были помещены в три пакета, горловины которых были перевязаны ниткой, концы которой оклеены бумажной биркой сотском круглой печати № 66 «Для пакетов» ОМВД России по Апшеронскому району, где он, понятые Н.В.П., Рякин Д.А. поставили свои подписи. Рякин пояснил, что имеющееся у него наркотическое средство, он не выдал добровольно, так как думал, что сотрудники полиции не будут осматривать летнюю кухню тщательно и наркотическое средство они не обнаружат, и о том, что хранить наркотические средства запрещено законом ему тоже было известно.
В судебном заседании по ходатайству государственного обвинителя, с согласия подсудимого, адвоката, в связи с неявкой в суд свидетеля К.В.В., в соответствии со ст. 281 УПК РФ оглашен протокол его допроса в качестве свидетеля от 06.03.2012 года (л.д. 79-81). Оглашенные показания свидетеля К.В.В. аналогичны с показаниями свидетеля Х.А.А.
В судебном заседании по ходатайству государственного обвинителя, с согласия подсудимого, адвоката, в связи с неявкой в суд свидетеля Н.В.П., в соответствии со ст. 281 УПК РФ оглашен протокол его допроса в качестве свидетеля от 06.03.2012 года (л.д. 38-39), из которого следует что он с 1982 года проживает по адресу <адрес>. У врачей нарколога и психиатра на учете не состоит. Так как он проживает один, к нему приходит сотрудник социальной службы и помогает по хозяйству, приносит продукты питания, иногда к нему приходят знакомые, с которыми он распивает спиртное. Раза два или три к нему приходили знакомы, приносили алкогольные напитки и некоторые продуты питания, и просили у него разрешения на изготовление и употребление наркотиков. Он разрешал им пользоваться своей посудой и газовой плитой, так как за это они давали ему спиртное, а иногда и продукты питания. 26 февраля 2012 года, днем к нему пришел Рякин, который попросил у него разрешения изготовить наркотик, при этом предложив ему бутылку пива, за что он разрешил Рякину изготовить наркотик и дал ему металлический ковш. Все ингредиенты для изготовления наркотика Рякин принес с собой, когда он изготавливал и употреблял наркотик Н.В.П. не видел, так как находился в другой комнате. В тот же день к нему домой приехали сотрудники полиции с понятыми и пояснили, что кто-то позвонил и сообщил о том, что у него дома находятся наркоманы и попросили разрешения на осмотр домовладения. Сотрудникам полиции Н.В.П. сообщил, что распивает пиво вместе с Рякиным, и позволил сотрудникам полиции осмотреть летнюю кухню, так как думал, что Рякин избавился от наркотиков, однако сотрудники полиции обнаружили наркотики и изъяли их в присутствии понятых, а Рякина Д.А. увезли на освидетельствование. 
Кроме показаний подсудимого, свидетелей вина подсудимого Рякина Д.А. подтверждается другими доказательствами:
 - рапортом об обнаружении признаков преступления составленный инспектором РППСМ Отдела МВД России по Апшеронскому району капитаном полиции Ф.Д.Н., согласно которому 26.02.2012 года в 14 часов 40 минут в ходе осмотра летней кухни домовладения <адрес> обнаружена бутылка с жидкостью светло-коричневого цвета с резким запахом ..., эмалированный ковш с жидкостью коричневого цвета, пластиковая бутылка с семенами растения мак, медицинский шприц, принадлежащие Рякину Д.А., ДД.ММ.ГГГГ года рождения. (л.д.6)
протоколом медицинского освидетельствования № от 26.02.2012 года согласно которому установлен факт нахождение Рякина Д.А. в состояния одурманивания наркотическим веществом (морфин) и алкогольное опьянение. (л.д.19)
протоколом об административном правонарушении № 000406 от 26.02.2012 года составленным на Рякина Д.А., ДД.ММ.ГГГГ года рождения по ст. 6.9 КоАП РФ. (л.д.20)
протоколом осмотра места происшествия с фото таблицами, согласно которому осмотренным местом происшествия является кухня домовладении <адрес>. (л.д. 9-10) 
 заключением эксперта № 83 - Э от 28.02.2012 года, согласно которому в ходе осмотра установлено, что жидкость коричневого цвета (объект № 2 и № 3), содержит наркотическое средство - экстракт маковой соломы. Масса экстракта маковой соломы, высушенная при температуре 110С, составляет 1,26 грам. (л.д.28-29) 
Согласно заключению эксперта № 109 - Э от 29.03.2012 года, (объекты № 2 и № 3), представленные на экспертизу по уголовному делу № 371175, содержит наркотическое средство-экстракт маковой соломы. Масса наркотического средства экстракта маковой соломы с учетом заключения эксперта № 83 -Э от 28.02.2012 года составляет 1,26 грамма высушенного при температуре 110 С.Наркотическое средство экстракт маковой соломы получают из маковой соломы любым способом, ...
Согласно заключению эксперта № 110-Э от 29.03.2012 года, вещество, представленное на исследование, по уголовному делу № 37175, является семенами растения мак, которые наркотических веществ не содержат и наркотическим средством не являются. Представленные семена ранее подвергались обработке .... (л.д. 70-71). 
Согласно заключению эксперта № 111-Э от 29.03.2012 года, в ходе которого было установлено, что налет коричневого цвета, на внутренней поверхности эмалированного ковша, представленного на экспертизу по уголовному делу № 371175, является наркотическим средством - экстракт маковой соломы. Масса наркотического средства экстракта маковой соломы, высушенного при температуре 110С, составляет 0,3 грамма (л.д.72-73). 
 Оценивая все обстоятельства дела в их совокупности, суд считает что действия Рякина Д.А. органами предварительного следствия, верно квалифицированы по ч. 1 ст. 228 УК РФ, как незаконное изготовление, хранении без цели сбыта наркотических средств в крупном размере.
По своему психическому состоянию Рякин может правильно воспринимать обстоятельства, имеющие значение для дела и давать показания. В ходе судебного следствия установлено, что во время совершения преступления Рякин действовал последовательно, целенаправленно, правильно ориентировался в окружающей обстановке и происходящих событиях, осознано руководил своими действиями. Его поведение в судебном заседании адекватно происходящему, он давал обдуманные и последовательные показания. Свою защиту осуществлял также обдумано, мотивировано, активно, у суда не возникло сомнений в его психической полноценности. Учитывая изложенные обстоятельства, суд находит, что Рякин является вменяемым и подлежит уголовной ответственности и наказанию.
С учетом всех доказательств по делу, с точки зрения относимости, допустимости и достоверности, суд признает их в совокупности достаточными для признания вины Рякину по ч. 1 ст. 228 УК РФ.
При назначении наказания суд учитывает, характер и степень общественной опасности совершенного преступления, данные о личности, обстоятельства дела, смягчающие и отягчающие наказание, а также влияние назначенного наказания на исправление осужденного.
Рякин совершил преступление, которое относится категории небольшой тяжести, ...». Согласно заключению экспертов от 03.04.2012 года № 605 Рякин Д.А. в принудительном лечении не нуждается и осознает фактический и общественный характер своих действий и руководит ими, как в момент совершения правонарушения так и в настоящее время.
К обстоятельствам, смягчающим наказание Рякину Д.А., суд относит признание вины в совершенном преступлении, раскаянье в содеянном, наличие на иждивении малолетнего ребенка, 2005 года рождения.
Обстоятельств, отягчающих наказание Рякину Д.А. по преступлению, судом не установлено.
С учетом всех обстоятельств дела, личности подсудимого, суд считает, что Рякину Д.А. должно быть назначено наказании в виде штрафа. 
На основании изложенного и руководствуясь ст.ст. 307-309 УПК РФ, суд 
 """

In [18]:
get_text_embedding(text)

array([ 3.33126548e-03, -2.19774431e-03, -3.77667320e-04,  2.98026497e-03,
        1.62544152e-03,  3.31497764e-03, -1.80126167e-03, -6.76269360e-04,
       -6.77539719e-04, -1.56332161e-03, -1.46665684e-04,  5.66154484e-04,
        1.68125913e-03, -2.14244389e-04,  7.81881398e-04,  2.96464989e-05,
       -6.06519002e-05,  1.25224175e-04, -5.49652341e-04,  1.01518774e-03,
       -1.52131596e-03, -2.67374425e-04,  1.54200562e-03, -2.86105064e-03,
        5.92843283e-04, -1.44571412e-03, -2.77883454e-05, -1.09178564e-03,
       -3.59463120e-04,  2.83132722e-03,  1.74301420e-04, -3.75416438e-06,
        1.04974989e-03, -2.14740461e-03,  6.91998793e-04, -1.02048887e-03,
       -2.17951894e-04, -1.65659636e-03,  2.40351779e-04,  1.47661078e-03,
       -9.04322521e-04,  1.07630380e-03,  6.63275003e-04,  2.53937112e-03,
        1.00524494e-03, -1.22439569e-04, -5.03521210e-04, -1.31480030e-04,
        9.33078068e-04, -2.48362969e-03,  8.24910285e-04,  1.45896696e-03,
       -1.48494736e-03, -

In [55]:
%%time
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
#with open('C:/Users/User/papaver_main_parts/main_3.txt', 'r', encoding='utf8') as file:
with open('C:/Users/User/papaver_main_parts/main_5.txt', 'r', encoding='utf8') as file:
    line = file.readline()
    #while line != '':
    for i in range(90000):
        pars = [l for l in line.split('\\n') if len(re.findall(r'[а-я]', l))>0]
        for par in pars:
            trainer.train(par)

Wall time: 180 ms


In [56]:
tokenizer = PunktSentenceTokenizer(trainer.get_params())
#with open('C:/Users/User/papaver_main_parts/main_3.txt', encoding='utf8') as file:
with open('C:/Users/User/papaver_main_parts/main_5.txt', encoding='utf8') as file:
    line = file.readline()
    pars = [l for l in line.split('\\n') if len(re.findall(r'[а-я]', l))>0]
    text = []
    for par in pars:
        text += tokenizer.tokenize(par)
    print(text)

[]


In [57]:
def split_text(line, border = 0.47):
    pars = [l for l in line.split('\\n') if len(re.findall(r'[а-я]', l))>0]
    text = []
    for par in pars:
        text += tokenizer.tokenize(par)
    num_sents = len(text)
    i=0
    while i<num_sents:
        try:
            if len(re.findall(r'[а-я]', text[i]))==0:
                if len(re.findall(r'\d*.', text[i]))>0:
                    text[i]=text[i]+text[i+1]
                    del text[i+1]
                    num_sents = num_sents - 1
                elif len(re.findall(r')', text[i]))>0:
                    if len(re.findall(r'\d*.', text[i]))>0:
                        text[i-1]=text[i-1]+text[i]
                        del text[i]
                        num_sents = num_sents - 1
            i+=1
        except:
            i+=1
    parts = []
    curr_part = text[0]
    for i in range(len(text)-1):
        sim = cosine_similarity(get_text_embedding(text[i]).reshape(1, -1),
                                get_text_embedding(text[i+1]).reshape(1, -1))[0][0]
        if sim>=border:
            curr_part = curr_part+'\n'+text[i+1]
        else:
            parts.append(curr_part)
            curr_part = text[i+1]
    parts.append(curr_part)
    del curr_part
    return parts

In [59]:
with open('C:/Users/User/papaver_main_parts/main_5.txt', encoding='utf8') as file:
#with open('C:/Users/User/papaver_main_parts/main_3.txt', encoding='utf8') as file:
    for line in range(2):
        line = file.readline()
    splitted = split_text(line, border = 0.15)
    print(len(splitted))
    for _ in splitted:
        print(_)
        print('\n\n')

52
Рякин Д.А., совершил незаконное изготовление, хранение без цели сбыта наркотических средств в крупном размере.



Преступления совершено им при следующих обстоятельствах.



Рякин Д.А., 26 февраля 2012 года около 14 часов, находясь в летней кухне <адрес>, из ранее приобретенных им семян мака и растворителя, умышленно, незаконно в нарушении требования ФЗ № 3 от 08.01.1998 года «О наркотических средствах и психотропных веществах», без цели сбыта, ..., изготовил средство, которое согласно заключений эксперта, является наркотическим средством - экстрактом маковой соломы, массой не менее 1,29 грамма сухого вещества, размер которого согласно утвержденного Постановления Правительства Российской Федерации № 76 от 07.02.2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, для целей статей 228, 228.1 и 229 Уголовного кодекса Российской Федерации», является крупным размер.
В тот же день часть незаконно изготовленного наркотического средства

In [79]:
print('ФАБУЛА')
print('\n')
print(splitted[2])
print('\n')
print(splitted[3])
print('\n')
print(splitted[4])
print('\n')
print(splitted[5])
print('\n')
print(splitted[6])
print('\n')
print(splitted[7])
print('\n')
print(splitted[8])

ФАБУЛА


Рякин Д.А., 26 февраля 2012 года около 14 часов, находясь в летней кухне <адрес>, из ранее приобретенных им семян мака и растворителя, умышленно, незаконно в нарушении требования ФЗ № 3 от 08.01.1998 года «О наркотических средствах и психотропных веществах», без цели сбыта, ..., изготовил средство, которое согласно заключений эксперта, является наркотическим средством - экстрактом маковой соломы, массой не менее 1,29 грамма сухого вещества, размер которого согласно утвержденного Постановления Правительства Российской Федерации № 76 от 07.02.2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, для целей статей 228, 228.1 и 229 Уголовного кодекса Российской Федерации», является крупным размер.
В тот же день часть незаконно изготовленного наркотического средства - экстракт маковой соломы Рякин Д.А.
употребил ..., а оставшуюся часть наркотического средства - экстракт маковой соломы массой 1,29 грамма, что относится к крупному р

In [85]:
print('ПОКАЗАНИЯ СВИДЕТЕЛЕЙ')
print('\n')
print(splitted[9])
print('\n')
print('...')
print('\n')
print(splitted[20])
print('\n')
print(splitted[21])
print('\n')
print(splitted[22])

ПОКАЗАНИЯ СВИДЕТЕЛЕЙ


Оценивая все доказательства в их совокупности, суд находит, что вина подсудимого Рякина Д.А.
в совершенном им преступлении установлена в судебном заседании и подтверждается следующими доказательствами.
В судебном заседании по ходатайству государственного обвинителя, с согласия подсудимого, адвоката, в связи с неявкой в суд свидетеля Х.А.А., в соответствии со ст.
281 УПК РФ оглашен протокол его допроса в качестве свидетеля от 12.03.2012 года (л.д.
35-37), из которого следует, что 26.02.2012 года около 14 часов 30 минут он находился в районе <адрес>, его остановили сотрудники полиции и пригласили в качестве понятого при осмотре домовладения <адрес>, после чего они пригласили еще одного понятого и разъяснили им права и обязанности понятых.
После чего сотрудник полиции пояснил, что в Дежурную Часть Отдела МВД России по Апшеронскому району поступило сообщение, что по данному адресу собрались наркоманы и изготавливают наркотические средства.
В его присутствии и в прису

In [103]:
print('ДОКАЗАТЕЛЬСТВА')
print('\n')
print(splitted[23])
print('\n')
print(splitted[24])
print('\n')
print(splitted[25])
print('\n')
print(splitted[26])
print('\n')
print('...')
print('\n')
print(splitted[29])

ДОКАЗАТЕЛЬСТВА


Кроме показаний подсудимого, свидетелей вина подсудимого Рякина Д.А.


подтверждается другими доказательствами: - рапортом об обнаружении признаков преступления составленный инспектором РППСМ Отдела МВД России по Апшеронскому району капитаном полиции Ф.Д.Н., согласно которому 26.02.2012 года в 14 часов 40 минут в ходе осмотра летней кухни домовладения <адрес> обнаружена бутылка с жидкостью светло-коричневого цвета с резким запахом ..., эмалированный ковш с жидкостью коричневого цвета, пластиковая бутылка с семенами растения мак, медицинский шприц, принадлежащие Рякину Д.А., ДД.ММ.ГГГГ года рождения.


(л.д.6) протоколом медицинского освидетельствования № от 26.02.2012 года согласно которому установлен факт нахождение Рякина Д.А.


в состояния одурманивания наркотическим веществом (морфин) и алкогольное опьянение.


...


(л.д.20) протоколом осмотра места происшествия с фото таблицами, согласно которому осмотренным местом происшествия является кухня домовладении <адрес>

In [110]:
print('ЭКСПЕРТНОЕ ЗАКЛЮЧЕНИЕ')
print('\n')
print(splitted[30])
print('\n')
print(splitted[31])
print('\n')
print(splitted[32])
print('\n')
print(splitted[33])
print('\n')
print('...')
print('\n')
print(splitted[36])

ЭКСПЕРТНОЕ ЗАКЛЮЧЕНИЕ


(л.д.


9-10) заключением эксперта № 83 - Э от 28.02.2012 года, согласно которому в ходе осмотра установлено, что жидкость коричневого цвета (объект № 2 и № 3), содержит наркотическое средство - экстракт маковой соломы.


Масса экстракта маковой соломы, высушенная при температуре 110С, составляет 1,26 грам.


(л.д.28-29)  Согласно заключению эксперта № 109 - Э от 29.03.2012 года, (объекты № 2 и № 3), представленные на экспертизу по уголовному делу № 371175, содержит наркотическое средство-экстракт маковой соломы.
Масса наркотического средства экстракта маковой соломы с учетом заключения эксперта № 83 -Э от 28.02.2012 года составляет 1,26 грамма высушенного при температуре 110 С.Наркотическое средство экстракт маковой соломы получают из маковой соломы любым способом, ... Согласно заключению эксперта № 110-Э от 29.03.2012 года, вещество, представленное на исследование, по уголовному делу № 37175, является семенами растения мак, которые наркотических веществ не сод

In [117]:
print('МЕДИТАЦИЯ СУДЬИ')
print('\n')
print(splitted[37])
print('\n')
print(splitted[38])
print('\n')
print(splitted[39])
print('\n')
print(splitted[40])
print('\n')
print(splitted[41])
print('\n')
print('...')
print('\n')
print(splitted[51])

МЕДИТАЦИЯ СУДЬИ


Оценивая все обстоятельства дела в их совокупности, суд считает что действия Рякина Д.А.


органами предварительного следствия, верно квалифицированы по ч.


1 ст.


228 УК РФ, как незаконное изготовление, хранении без цели сбыта наркотических средств в крупном размере.


По своему психическому состоянию Рякин может правильно воспринимать обстоятельства, имеющие значение для дела и давать показания.
В ходе судебного следствия установлено, что во время совершения преступления Рякин действовал последовательно, целенаправленно, правильно ориентировался в окружающей обстановке и происходящих событиях, осознано руководил своими действиями.
Его поведение в судебном заседании адекватно происходящему, он давал обдуманные и последовательные показания.
Свою защиту осуществлял также обдумано, мотивировано, активно, у суда не возникло сомнений в его психической полноценности.


...


должно быть назначено наказании в виде штрафа.
